# **PYTHON-13. Визуализация данных**

Примечание: во многих заданиях сознательно применена гистограмма, а не столбцы, по причине возможности отражения значений в процентах в гистограммах, чего нельзя сделать в столбчатых диграммах.

##Проект: разведывательный анализ данных по оттоку клиентов банка. 

Загрузка данных из churn.csv

In [18]:
!pip install plotly
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
churn_data = pd.read_csv('/content/churn.csv')
#
churn_data = churn_data.drop('RowNumber', axis=1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Задание 1

Каково соотношение ушедших и лояльных клиентов? Покажите это на графике и дайте комментарий по соотношению.

In [19]:
exited_data = churn_data.groupby(
    by='Exited',
    as_index=False
)['Exited'].count()


In [20]:
#строим график
fig = px.pie(exited_data, values='Exited', 
             names=['Лояльные клиенты', 'Ушедшие клиенты'], width=600,
             title='Cоотношение ушедших и лояльных клиентов')
#отображаем его
fig.show()

Указанное соотношение наглядней всего показать на круговой диаграмме. Как видно, ушедшие клиенты составляют пятую часть от общего числа бывших и имеющихся клиентов банка. Кроме того для полноты картины желательно иметь данные об оттоке в ретроспективе (20% - это за год или 10 лет), но эти данные отсутствуют.

### Задание 2

Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов. Опишите распределение и сделайте выводы.

In [21]:
balance_data = churn_data[churn_data['Balance'] > 2500]
fig = px.histogram(balance_data, x='Balance', y='CustomerId', nbins=12, histnorm = "percent", 
                   text_auto='.0f', width=600, title='Распределение баланса клиентов')
fig.show()

Для визуализации распределения размера балансов клиентов использовалась гистограмма в процентах. При количестве корзин равном 12 получился  несимметричный относительно максимума график. Наибольший объем приходиться на балансы с величиной от 100 тысяч до 150 тысяч. С точки зрения охвата есть большой потенциал по привлечению клиентов в нишу до 50 тыс. и выше 200 тыс.

### Задание 3

Посмотрите на распределение баланса клиента в разрезе признака оттока. Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

In [22]:
fig = px.histogram(balance_data, x='Balance', y='CustomerId', color='Exited', nbins=12, text_auto='.0f', width=600, histfunc='count', 
                   title='Распределение баланса клиентов')
fig.show()

Гистограмма показывает, что наибольшая доля ушедших клиентов приходится как раз на столбец с наибольшим объёмом балансов. Объяснением может быть то что клиенты этой категории имеют более активную жизненную позицию чем клиенты других категорий предпочитающие оставаться лояльными банку.

### Задание 4

Посмотрите на распределение возраста в разрезе признака оттока. В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

In [23]:
age_exited_data = churn_data[(churn_data['Balance'] > 2500) & (churn_data['Exited']==1)]
fig = px.histogram(balance_data, x='Age', color='Exited', nbins=12, histnorm = "percent", width=600,
                   text_auto='.0f', title='Распределение возраста ушедших клиентов')
fig.show()

fig = px.box(balance_data, x='Age', color='Exited', boxmode='group', 
                title='Распределение возраста ушедших клиентов')

fig.show()

График показывает четко выраженную зависимость доли ушедших клиентов от их возраста: от 6% в группе 20-30 лет до 40% всех ушедших в группе от 40-50  лет, на эту группу лучше всего сфокусировать внимание банку по причине того что люди в этом возрасте считаются состоявшимися. Количество выбросов значительно выше в группе оставшихся клиентов.

### Задание 5

Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. Добавьте расцветку по признаку оттока клиентов. Какова взаимосвязь между признаками? Если не видите явной взаимосвязи, укажите это.

In [24]:
fig = px.scatter(balance_data, x='CreditScore', y='EstimatedSalary', 
                 color='Exited', log_y=True, width=500, title='Зависимость рейтинга клиента от его зарплаты')
fig.show()

На линенйном графике зависимости кредитного рейтинга клиента и его  зарплаты сильный разброс значений не позоляет выявить какую-либо взаимосвязь. Применив логрифмирование к шкале зарплаты можно выделить скученность значений с явно выраженнной ложбиной с минимумом при рейтинге 650, это значение приблизительно делит всю совокупность данных пополам, причем ухудшение или улучшение рейтинга относительно этого значения одинаково свойственно клиентам как с высоким так и с низким уровнем предполагаемой зарплаты (возможно предположение неверно) и практически не зависит от признака оттока, не считая 10 ушедших клиентов с рейтингом ниже 400 и зарплатой выше 50 тыс.

### Задание 6

Кто чаще уходит, мужчины или женщины? Постройте график, который иллюстрирует это.

In [25]:
gender_data = churn_data.groupby(
    by=['Gender', 'Exited'],
    as_index=False
)['CustomerId'].count()
gender_data['Exited'] = gender_data['Exited'].astype('category')
fig = px.histogram(gender_data, x='Gender', y='CustomerId', color='Exited', histnorm = "percent", 
                   text_auto='.0f', width=600, title='Соотношение долей ушедших мужчин и женщин')
fig.show()

Доля ушедших клиентов среди женщин выше, чем доля ушедших мужчин на 12%.

### Задание 7

 Как отток клиентов зависит от числа приобретённых у банка услуг? Для ответа на этот вопрос постройте многоуровневую столбчатую диаграмму.

In [26]:
products_data = churn_data.groupby(
    by=['NumOfProducts', 'Exited'],
    as_index=False
)['CustomerId'].count()
products_data['NumOfProducts'] = products_data['NumOfProducts'].astype('category')
products_data = products_data[products_data['Exited'] == 1]
fig = px.bar(products_data, x='Exited', y='CustomerId', color='NumOfProducts', width=600,  text_auto='.0f', title='Зависимость оттока клиентов от числа услуг')
fig.show()

Практически 70% ушедших клиентов были подписаны на одну услугу 17% - на две и 11% - на 3. Это говорит что большее количество услуг побуждает людей оставаться клиентами банка.

### Задание 8

Как влияет наличие статуса активного клиента на отток клиентов? Постройте диаграмму, иллюстрирующую это. Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?

In [27]:
activ_data = churn_data.groupby(
    by=['IsActiveMember', 'Exited'],
    as_index=False
)['CustomerId'].count()
activ_data['IsActiveMember'] = activ_data['IsActiveMember'].astype('category')
#activ_data = activ_data[activ_data['Exited'] == 1]
fig = px.histogram(activ_data,  x='Exited', y='CustomerId', color='IsActiveMember', 
                   histnorm = "percent", width=600, text_auto='.0f', title='Зависимость оттока клиентов от статуса активности')
fig.show()

Статус активного клиента существенно уменьшает вероятность ухода. Потому для снижения риска ухода банку следует поощрять активность клиентов в совершении операций.

### Задание 9

В какой стране доля ушедших клиентов больше? Постройте тепловую картограмму, которая покажет это соотношение на карте мира. Предположите, с чем это может быть связано.

In [28]:
geography_data = churn_data.groupby(
    by='Geography',
    as_index=False
)['Exited'].mean()
geography_data['Exited'] = geography_data['Exited']  * 100
fig = px.choropleth(
    data_frame=geography_data, #DataFrame
    locations="Geography", #столбец с локациями
    locationmode = "country names", #режим сопоставления локаций с базой Plotly
    color="Exited", #от чего зависит цвет
 
    range_color=[0, 100], #диапазон цвета
    title='Распределение доли ушедших клиентов по странам присутствия банка', #заголовок
    width=800, #ширина
    height=500, #высота
    color_continuous_scale='Reds' #палитра цветов
)
fig.show()

В Германии доля ушедших клиентов выше чем в Испании или Франции в два раза. Предпложительно это стало следствием наложения всех вышеперечисленых факторов: в Германии выше доля активного населения возрастом от 40 до 50 лет с балансом выше среднего для которого важно просто наличие баланса но при этом этот баланс не предназначен для текущих операций.

### Задание 10

Найдите на тепловой карте категории клиентов, которые уходят чаще всего.

In [29]:
def get_credit_score_cat(credit_score):
    if credit_score >= 300 and credit_score < 500:
        return "Very_Poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score >= 851:
        return "Top"
    elif credit_score < 300:
        return "Deep"

churn_data['CreditScoreCat'] = churn_data['CreditScore'].apply(get_credit_score_cat).astype('category')
pivot_date = churn_data.pivot_table(
    values='Exited',
    #index=C,
    columns=['Tenure', 'CreditScoreCat']
    #fill_value=0,
    #aggfunc='mean'
)

pivot_date = pd.melt(pivot_date, var_name=['Tenure', 'CreditScoreCat'], value_name="Exited")
fig = px.density_heatmap(
    pivot_date, #DataFrame
    x="Tenure",
    y="CreditScoreCat",
    z='Exited',
    histfunc='avg',
    nbinsx=11
)
fig.show()

На диаграмме четко видно что наихудшим рейтингом обладает категория клиентов на первом году своего нахождения в банке с долей ушедших 44%.